                                    ETL PARA output_steam_games.json

In [10]:
import pandas as pd
import json
%load_ext autoreload
%autoreload 2
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


                LECTURA Y EXTRACCION DE LOS DATOS EN EL ARCHIVO output_steam_games.json

In [11]:
#Cargar el archivo json
output_games = '../Json iniciales/output_steam_games.json'


#Leer el dataset 
filas = []
with open (output_games, 'r', encoding='utf-8') as f:
    for line in f.readlines():
        data = json.loads(line)
        filas.append(data)

#Convertir a dataframe
df_output_games = pd.DataFrame(filas)
df_output_games.head(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Se aprecian ciertas filas con registros vacíos, vamos a deshacernos estos:


In [12]:
df_output_games = df_output_games.dropna(how='all').reset_index(drop=True)
df_output_games.shape

(32135, 13)

Se observa el dataframe resultante:

In [13]:
df_output_games.head(5)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN


In [14]:
df_output_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

Algunas columnas no nos serán útiles teniendo en cuenta los endpoints solicitados, por ejemplo
Publisher y Developer, representan la misma informacion, al igual que app_name y Title, otras columnas son poco útiles, como url y reviews_url.

In [15]:
df_output_games = df_output_games.drop(['tags', 'specs', 'url', 'reviews_url'], axis=1)
df_output_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'release_date', 'price',
       'early_access', 'id', 'developer'],
      dtype='object')

Se renombran las columnas:

In [16]:

nuevos_nombres = {'publisher': 'Publisher', 'genres': 'Genres','app_name':'App_Name', 'title': 'Title',
                  'release_date':'Release_Date','price':'Price',
                  'early_access':'Early_Access','id':'Item_Id','developer':'Developer'}
df_output_games = df_output_games.rename(columns=nuevos_nombres)

df_output_games.columns

Index(['Publisher', 'Genres', 'App_Name', 'Title', 'Release_Date', 'Price',
       'Early_Access', 'Item_Id', 'Developer'],
      dtype='object')

                                        VERIFICAR SI EXISTEN DUPLICADOS:
                                        
Se verifican datos duplicados mediantela columna de Id_Item

In [17]:
#Creamos al funcion duplicados, toma nuestro dataframe, y la columna en la que se buscarán los duplicados:
def duplicados(dataframe, columna):
    #Buscamos los duplicados en la columna y los guardamos, si no existen, retornamos 'sin duplicados'
    duplicados = dataframe[dataframe.duplicated(subset=columna, keep=False)]
    if duplicados.empty:
        return "Sin duplicados"

    duplicados_ordenados = duplicados.sort_values(by=columna)
    return duplicados_ordenados

duplicados(df_output_games,'Item_Id')

,Publisher,Genres,App_Name,Title,Release_Date,Price,Early_Access,Item_Id,Developer
13894,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,2017-10-26,59.99,False,612880,Machine Games
14573,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,2017-10-26,59.99,False,612880,Machine Games
74,NaN,NaN,NaN,NaN,NaN,19.99,False,NaN,NaN
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,2012-09-07,19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


Tenemos aquí dos pares de duplicados, por un lado, el Id_Item 612880, por otro lado, no se conocen los Id's del segundo par de duplicados, por lo que buscaremos a través de la columna developer para buscar mas registros de este producto:


In [18]:
df_output_games[df_output_games['Developer']=='Rocksteady Studios,Feral Interactive (Mac)']

,Publisher,Genres,App_Name,Title,Release_Date,Price,Early_Access,Item_Id,Developer
1068,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,2012-09-07,19.99,False,200260,"Rocksteady Studios,Feral Interactive (Mac)"
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,2012-09-07,19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"
31617,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham Asylum Game of the Year Edition,Batman: Arkham Asylum Game of the Year Edition,2010-03-26,19.99,False,35140,"Rocksteady Studios,Feral Interactive (Mac)"


Por lo tanto, el Id_Item correspoindiente, en este caso es 200260, procedemos a eliminar los duplicados:

In [19]:
eliminar_duplicados = [14573, 74]
df_output_games = df_output_games.drop(eliminar_duplicados)

In [20]:
#verificamos duplicados:
duplicados(df_output_games,'Item_Id')

'Sin duplicados'

                                            COLUMNA RELEASE_YEAR:

Teniendo en cuenta que en las funciones para la API, solo será necesario el año de lanzamiento de cada juego, se crea una nueva columna 'Release_Year', posteriormente se elimina la columna Release_Date:

In [21]:
#Se trabajará sobre la columna Release_Date, convirtiendo a formato de fecha:
df_output_games['Release_Date'] = pd.to_datetime(df_output_games['Release_Date'],dayfirst=True, errors='coerce')

In [22]:
#Se remueven los datos que no esten en formato fecha NaT:
df_output_games.dropna(subset = ['Release_Date'], inplace=True)

In [23]:
#Se verifica que el formato sea año-mes-día
df_output_games['Release_Date'] = pd.to_datetime(df_output_games['Release_Date']).dt.strftime('%Y-%m-%d')

#Se extrae solo el año, de la nueva columna Release Year
df_output_games['Release_Year'] = pd.to_datetime(df_output_games['Release_Date']).dt.year

#Se dropea la columna Release Date, y observamos la nueva columna Release Year.
df_output_games =df_output_games.drop(["Release_Date"], axis=1)
df_output_games[['Release_Year']]

,Release_Year
0,2018
1,2018
3,2017
5,2018
6,2018
...,...
32129,2018
32130,2018
32131,2018
32132,2018


In [24]:
#Todos los años que han tenido lanzamientos
df_output_games['Release_Year'].unique()

array([2018, 2017, 1998, 2006, 2005, 2002, 2007, 1995, 1994, 1993, 2003,
       2008, 1997, 2009, 1999, 1992, 1989, 2001, 2000, 2010, 2011, 2012,
       2013, 2004, 2015, 2014, 1996, 2016, 1990, 1988, 1991, 1985, 1987,
       1984, 1983, 1981, 1986, 2019, 1980, 1982])

In [25]:
#Dataframe resultante luego de obtener los años de lanzamiento:
df_output_games.head(5)

,Publisher,Genres,App_Name,Title,Price,Early_Access,Item_Id,Developer,Release_Year
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,Free To Play,False,643980,Secret Level SRL,2018
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,0.99,False,767400,彼岸领域,2017
5,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,3.99,False,772540,Trickjump Games Ltd,2018
6,NaN,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Basic Pass,SNOW - All Access Basic Pass,9.99,False,774276,Poppermost Productions,2018


Se trabaja con algunas columnas para asegurarnos que no contengan datos erroneos o nulos, ademas de modificar el tipo de dato si es necesario.

In [26]:
def tipo_de_Dato_df(dataframe):
    datos = {"Columna": [], "tipo_de_dato": [], "%_no_nulos": [], "%_nulos": [], "nulos": []}

    for columna in dataframe.columns:
        no_nulos = (dataframe[columna].count() / len(dataframe)) * 100
        datos["Columna"].append(columna)
        datos["tipo_de_dato"].append(dataframe[columna].apply(type).unique())
        datos["%_no_nulos"].append(round(no_nulos, 2))
        datos["%_nulos"].append(round(100-no_nulos, 2))
        datos["nulos"].append(dataframe[columna].isnull().sum())

    df_info = pd.DataFrame(datos)
        
    return df_info

tipo_de_Dato_df(df_output_games)

,Columna,tipo_de_dato,%_no_nulos,%_nulos,nulos
0,Publisher,"[<class 'str'>, <class 'float'>]",79.26,20.74,2363
1,Genres,"[<class 'list'>, <class 'float'>]",95.35,4.65,530
2,App_Name,[<class 'str'>],100.00,0.00,0
3,Title,[<class 'str'>],100.00,0.00,0
4,Price,"[<class 'float'>, <class 'str'>]",96.66,3.34,380
5,Early_Access,[<class 'bool'>],100.00,0.00,0
6,Item_Id,"[<class 'str'>, <class 'float'>]",99.99,0.01,1
7,Developer,"[<class 'str'>, <class 'float'>]",95.15,4.85,553
8,Release_Year,[<class 'int'>],100.00,0.00,0


                                                Columna Publisher:

se rellena los Nulos de publisher con los datos en developer, y los datos faltantes de developer, con los datos de pusblisher, si no existen datos en ambas columnas, se dropea;

In [27]:
#Se llena nulos en 'Developer' con valores de 'Publisher'
df_output_games['Developer'].fillna(df_output_games['Publisher'], inplace=True)
#Se llena nulos en 'Publisher' con valores de 'Developer'
df_output_games['Publisher'].fillna(df_output_games['Developer'], inplace=True)
#Se elimina filas donde ambas columnas 'Developer' y 'Publisher' son nulas
df_output_games.dropna(subset=['Developer', 'Publisher'], how='all', inplace=True)


In [28]:
nulos_pub = df_output_games['Publisher'].isnull().sum()
nulos_pub

0

In [29]:
#Publisher debe ser de tipo string
df_output_games['Publisher'] = df_output_games['Publisher'].astype(str)

In [30]:
df_output_games.head(5)

,Publisher,Genres,App_Name,Title,Price,Early_Access,Item_Id,Developer,Release_Year
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,Free To Play,False,643980,Secret Level SRL,2018
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,0.99,False,767400,彼岸领域,2017
5,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,3.99,False,772540,Trickjump Games Ltd,2018
6,Poppermost Productions,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Basic Pass,SNOW - All Access Basic Pass,9.99,False,774276,Poppermost Productions,2018


                                                    Columna Genres:

Como vimos en a tabla anterior esta columna contiene elementos tipo lista, por lo que pasaremos estos elementos a una cadena de texto :

In [31]:
df_output_games['Genres']

0        [Action, Casual, Indie, Simulation, Strategy]
1                 [Free to Play, Indie, RPG, Strategy]
3                          [Action, Adventure, Casual]
5                      [Action, Adventure, Simulation]
6            [Free to Play, Indie, Simulation, Sports]
                             ...                      
32129               [Action, Adventure, Casual, Indie]
32130            [Casual, Indie, Simulation, Strategy]
32131                        [Casual, Indie, Strategy]
32132                      [Indie, Racing, Simulation]
32133                                  [Casual, Indie]
Name: Genres, Length: 10871, dtype: object

In [32]:
tipo_de_Dato_df(df_output_games)

,Columna,tipo_de_dato,%_no_nulos,%_nulos,nulos
0,Publisher,[<class 'str'>],100.00,0.00,0
1,Genres,"[<class 'list'>, <class 'float'>]",99.48,0.52,56
2,App_Name,[<class 'str'>],100.00,0.00,0
3,Title,[<class 'str'>],100.00,0.00,0
4,Price,"[<class 'float'>, <class 'str'>]",96.58,3.42,372
5,Early_Access,[<class 'bool'>],100.00,0.00,0
6,Item_Id,"[<class 'str'>, <class 'float'>]",99.99,0.01,1
7,Developer,[<class 'str'>],100.00,0.00,0
8,Release_Year,[<class 'int'>],100.00,0.00,0


In [33]:
df_output_games.dropna(subset=['Genres'], inplace=True)

In [34]:
df_output_games['Genres'] = df_output_games['Genres'].apply(lambda x: ','.join(x))
df_output_games = df_output_games.explode('Genres')

In [35]:
tipo_de_Dato_df(df_output_games)

,Columna,tipo_de_dato,%_no_nulos,%_nulos,nulos
0,Publisher,[<class 'str'>],100.00,0.00,0
1,Genres,[<class 'str'>],100.00,0.00,0
2,App_Name,[<class 'str'>],100.00,0.00,0
3,Title,[<class 'str'>],100.00,0.00,0
4,Price,"[<class 'float'>, <class 'str'>]",96.65,3.35,362
5,Early_Access,[<class 'bool'>],100.00,0.00,0
6,Item_Id,"[<class 'str'>, <class 'float'>]",99.99,0.01,1
7,Developer,[<class 'str'>],100.00,0.00,0
8,Release_Year,[<class 'int'>],100.00,0.00,0


                                                    Columna Price:


Al ir intentando convertir FreeToPlay como $0.0, saltan errores con nombres similares, por ejemplo Free, Free Demo, Play For free!, etc, todos estos valores seran reemplazados por $0

In [36]:

df_output_games['Price'].replace('Free To Play', 0, inplace=True)
df_output_games['Price'].replace('Free to Play', 0, inplace=True)
df_output_games['Price'].replace('Free', 0, inplace=True)
df_output_games['Price'].replace('Free Demo', 0, inplace=True)
df_output_games['Price'].replace('Play for Free!', 0, inplace=True)
df_output_games['Price'].replace('Install Theme', 0, inplace=True)
df_output_games['Price'].replace('Third Party', 0, inplace=True)
df_output_games['Price'].replace('Third-party', 0, inplace=True)
df_output_games['Price'].replace('Free HITMAN™ Holiday Pack', 0, inplace=True)
df_output_games['Price'].replace('Play the Demo', 0, inplace=True)
df_output_games['Price'].replace('Play the Demo', 0, inplace=True)
df_output_games['Price'].replace('Starting at $499.00', 49.9, inplace=True)
df_output_games['Price'].replace('Starting at $449.00', 44.9, inplace=True)
df_output_games['Price'].replace('Free to Try', 44.9, inplace=True)
#Pasar a float
df_output_games['Price'] = df_output_games['Price'].astype(float)


In [37]:
#Se buscan y eliminan nulos:
nulos_price = df_output_games['Price'].isnull().sum()
nulos_price

362

In [38]:
df_output_games.dropna(subset = ['Price'], inplace=True)
tipo_de_Dato_df(df_output_games)

,Columna,tipo_de_dato,%_no_nulos,%_nulos,nulos
0,Publisher,[<class 'str'>],100.00,0.00,0
1,Genres,[<class 'str'>],100.00,0.00,0
2,App_Name,[<class 'str'>],100.00,0.00,0
3,Title,[<class 'str'>],100.00,0.00,0
4,Price,[<class 'float'>],100.00,0.00,0
5,Early_Access,[<class 'bool'>],100.00,0.00,0
6,Item_Id,"[<class 'str'>, <class 'float'>]",99.99,0.01,1
7,Developer,[<class 'str'>],100.00,0.00,0
8,Release_Year,[<class 'int'>],100.00,0.00,0


Columna Developer


La columna developer debe quedar como string,por lo tanto se normaliza y se cambia su tipo

In [39]:
#Primero se normaliza
def normalizar_developer(desarrollador):
    if isinstance(desarrollador, str):  # Verificar si es una cadena antes de normalizar
        dev_limpio = re.sub(r'[^a-zA-Z\s]', '', desarrollador)
        dev_normalizado = re.sub(r'\s+', ' ', dev_limpio.lower().strip())
        return dev_normalizado
    else:
        return desarrollador

df_output_games['Developer'] = df_output_games['Developer'].apply(normalizar_developer)

#Se pasa a str
df_output_games['Developer'] = df_output_games['Developer'].str.title()



In [40]:
nulos_dev = df_output_games['Developer'].isnull().sum()
nulos_dev

0

In [41]:
tipo_de_Dato_df(df_output_games)

,Columna,tipo_de_dato,%_no_nulos,%_nulos,nulos
0,Publisher,[<class 'str'>],100.00,0.00,0
1,Genres,[<class 'str'>],100.00,0.00,0
2,App_Name,[<class 'str'>],100.00,0.00,0
3,Title,[<class 'str'>],100.00,0.00,0
4,Price,[<class 'float'>],100.00,0.00,0
5,Early_Access,[<class 'bool'>],100.00,0.00,0
6,Item_Id,"[<class 'str'>, <class 'float'>]",99.99,0.01,1
7,Developer,[<class 'str'>],100.00,0.00,0
8,Release_Year,[<class 'int'>],100.00,0.00,0


In [42]:
df_output_games['Item_Id'] = df_output_games['Item_Id'].replace(pd.NA, np.nan)

#Dropear nulos
df_output_games.dropna(subset = ['Item_Id'], inplace=True)
df_output_games['Item_Id']


0        761140
1        643980
3        767400
5        772540
6        774276
          ...  
32129    745400
32130    773640
32131    733530
32132    610660
32133    658870
Name: Item_Id, Length: 10452, dtype: object

In [43]:
tipo_de_Dato_df(df_output_games)

,Columna,tipo_de_dato,%_no_nulos,%_nulos,nulos
0,Publisher,[<class 'str'>],100.0,0.0,0
1,Genres,[<class 'str'>],100.0,0.0,0
2,App_Name,[<class 'str'>],100.0,0.0,0
3,Title,[<class 'str'>],100.0,0.0,0
4,Price,[<class 'float'>],100.0,0.0,0
5,Early_Access,[<class 'bool'>],100.0,0.0,0
6,Item_Id,[<class 'str'>],100.0,0.0,0
7,Developer,[<class 'str'>],100.0,0.0,0
8,Release_Year,[<class 'int'>],100.0,0.0,0


In [44]:
df_output_games

,Publisher,Genres,App_Name,Title,Price,Early_Access,Item_Id,Developer,Release_Year
0,Kotoshiro,"Action,Casual,Indie,Simulation,Strategy",Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
1,"Making Fun, Inc.","Free to Play,Indie,RPG,Strategy",Ironbound,Ironbound,0.00,False,643980,Secret Level Srl,2018
3,彼岸领域,"Action,Adventure,Casual",弹炸人2222,弹炸人2222,0.99,False,767400,,2017
5,Trickjump Games Ltd,"Action,Adventure,Simulation",Battle Royale Trainer,Battle Royale Trainer,3.99,False,772540,Trickjump Games Ltd,2018
6,Poppermost Productions,"Free to Play,Indie,Simulation,Sports",SNOW - All Access Basic Pass,SNOW - All Access Basic Pass,9.99,False,774276,Poppermost Productions,2018
...,...,...,...,...,...,...,...,...,...
32129,Bidoniera Games,"Action,Adventure,Casual,Indie",Kebab it Up!,Kebab it Up!,1.99,False,745400,Bidoniera Games,2018
32130,Ghost_RUS Games,"Casual,Indie,Simulation,Strategy",Colony On Mars,Colony On Mars,1.99,False,773640,Nikita Ghostrus,2018
32131,Sacada,"Casual,Indie,Strategy",LOGistICAL: South Africa,LOGistICAL: South Africa,4.99,False,733530,Sacada,2018
32132,Laush Studio,"Indie,Racing,Simulation",Russian Roads,Russian Roads,1.99,False,610660,Laush Dmitriy Sergeevich,2018


In [45]:
df_output_games = df_output_games.drop(['Title', 'Early_Access'], axis=1)
df_output_games['Item_Id'] = df_output_games['Item_Id'].astype('int')
tipo_de_Dato_df(df_output_games)

,Columna,tipo_de_dato,%_no_nulos,%_nulos,nulos
0,Publisher,[<class 'str'>],100.0,0.0,0
1,Genres,[<class 'str'>],100.0,0.0,0
2,App_Name,[<class 'str'>],100.0,0.0,0
3,Price,[<class 'float'>],100.0,0.0,0
4,Item_Id,[<class 'int'>],100.0,0.0,0
5,Developer,[<class 'str'>],100.0,0.0,0
6,Release_Year,[<class 'int'>],100.0,0.0,0


Guardado del archivo output_games:

In [46]:
output_games_limpio = '../Parquets/ETL_steam_games.parquet'

df_output_games.to_parquet(output_games_limpio, index=False)

In [48]:
output_games_limpio = '../CSV/ETL_steam_games.csv'

df_output_games.to_csv(output_games_limpio, index=False, encoding = 'utf-8')